# Lecture 5 Recap
---

## 1. Minimum Variance Frontier
*   Minimum-variance frontier shows the risk-return opportunities available to the investor.
*   Efficient frontier
    *   There is an efficient portfolio for each risk level.
    *   Portfolios on the efficient frontier represent trade-offs in terms of risk and return.



<center>
<img src="../BFIN_Tutorial 3/Picture7.png" width="600"/>


## 2. Capital Allocation: Risk-free and Risky Assets
### Complete portfolio consists of the risky asset and the risk-free asset.
  
  * A portfolio risk closer to the desired level can be achieved by **changing the weights** of risk-free asset and risky assets in the complete portfolio.

$$
      W_{rf} + W_{risky} = 1
$$

$$
      E(r_p) = (1 - W_{risky}) \cdot r_f + W_{risky} \cdot E(r_{risky})
$$

$$
      E(r_p) = r_f + W_{risky} \cdot [E(r_{risky}) - r_f]
$$

$$
      \sigma^2(r_p) = w_{rf}^2 \cdot \sigma^2(r_f) + W_{risky}^2 \cdot \sigma^2(r_{risky}) + 2 \cdot W_{rf}W_{risky}Cov(r_f, r_{risky})
$$

* Since $\sigma^2(r_f) = 0$ and $Cov(r_f, r_{risky}) = 0$,

$$
      \sigma^2(r_p) = W_{risky}^2 \cdot \sigma^2(r_{risky})
$$


<center>
<img src="../BFIN_Tutorial 3/Picture2.png" width="500"/>

- Investors choose a point on the CAL based on their risk preference, using their indifference curves.
- Target: **highest possible utility**.

<center>
<img src="../BFIN_Tutorial 3/Picture3.png" width="700"/>

### Calculating Portfolio Expected Returns Using Standard Deviations
$$
E(r_p) = r_f + \left[ \frac{E(r_{risky}) - r_f}{\sigma(r_{risky})} \right] \cdot \sigma(r_p)
$$

* **Example**: Suppose the risk-free rate is 3.5%, the expected return and the standard deviation of the risky portfolio are 9% and 12%, respectively. What is portfolio's expected return?

## 3. The Investment Decision
- Top-down process with 3 steps:
  
  * Capital allocation: risky portfolio and risk-free asset
  
  * Asset allocation: across broad asset classes
  
  * Security selection: individual assets within an asset class


- **Finding the Optimal Risky Portfolio**
  * Maximizing the slope of the CAL (**Sharpe Ratio**)

  $$
      Sharpe Ratio = \frac {[E(r_{P}) - r_f]}{\sigma(r_{P})}
  $$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown, Math

# --- 1. Define Asset Parameters ---
E_rD, std_D = 0.08, 0.12  # Debt Fund
E_rE, std_E = 0.13, 0.20  # Equity Fund
correlation_DE = 0.30

# --- 2. Core Calculation Functions ---
def portfolio_performance(w_E):
    w_D = 1 - w_E
    E_rp = w_D * E_rD + w_E * E_rE
    cov_DE = correlation_DE * std_D * std_E
    var_p = w_D**2 * std_D**2 + w_E**2 * std_E**2 + 2 * w_D * w_E * cov_DE
    return E_rp, np.sqrt(var_p)

# --- 3. Combined Plotting and Info Function ---
def generate_output(weight_in_equity_pct, rf_rate_pct):
    weight_in_equity = weight_in_equity_pct / 100
    rf_rate = rf_rate_pct / 100
    
    # --- Calculate portfolio values ---
    E_r_selected, std_selected = portfolio_performance(weight_in_equity)
    
    # Find the Optimal Risky Portfolio (Tangency Portfolio)
    weights_E_frontier = np.linspace(0, 1, 100)
    frontier_returns, frontier_stds = np.array([portfolio_performance(w) for w in weights_E_frontier]).T
    sharpe_ratios = (frontier_returns - rf_rate) / frontier_stds
    optimal_idx = np.nanargmax(sharpe_ratios)
    E_r_optimal, std_optimal = frontier_returns[optimal_idx], frontier_stds[optimal_idx]
    sharpe_optimal = (E_r_optimal - rf_rate) / std_optimal
    
    # --- a. Info Panel (Markdown + MathJax) ---
    info_md = f"""
#### Asset Profiles:
- Debt Fund: E(r) = {E_rD:.2%}, σ = {std_D:.2%}
- Equity Fund: E(r) = {E_rE:.2%}, σ = {std_E:.2%}

#### Selected Portfolio ({weight_in_equity_pct:.0f}% Stocks):
- E(rₚ) = {E_r_selected:.2%}
- σₚ = {std_selected:.2%}

#### Optimal Risky Portfolio (P):
- E(rₚ) = {E_r_optimal:.2%}
- σₚ = {std_optimal:.2%}
- Sharpe Ratio = {sharpe_optimal:.2f}

#### Objective:
Maximize the slope of the CAL (Sharpe Ratio):
"""
    with output_info:
        clear_output(wait=True)
        display(Markdown(info_md))
        display(Math(r"S_P = \frac{E(r_p) - r_f}{\sigma_p}"))

    # --- b. Plot Generation ---
    with output_plot:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(7.5, 7.5))
        ax.plot(frontier_stds * 100, frontier_returns * 100, color='deepskyblue', linewidth=2.5)

        sharpe_selected = (E_r_selected - rf_rate) / std_selected if std_selected > 0 else 0
        cal_selected_x = np.array([0, 0.25]); cal_selected_y = rf_rate + sharpe_selected * cal_selected_x
        ax.plot(cal_selected_x * 100, cal_selected_y * 100, color='black', linewidth=1.5)
        ax.scatter(std_selected * 100, E_r_selected * 100, facecolors='none', edgecolors='black', s=100, zorder=5, linewidth=1.5)

        cal_optimal_x = np.array([0, 0.25]); cal_optimal_y = rf_rate + sharpe_optimal * cal_optimal_x
        ax.plot(cal_optimal_x * 100, cal_optimal_y * 100, color='black', linewidth=1.5)
        ax.scatter(std_optimal * 100, E_r_optimal * 100, color='black', s=100, zorder=5)

        points = {'A': portfolio_performance(0.18), 'B': portfolio_performance(0.30), 'E': portfolio_performance(0.82)}
        for label, (ret, std) in points.items():
            ax.scatter(std * 100, ret * 100, facecolors='none', edgecolors='black', s=100, zorder=5, linewidth=1.5)
            ax.text(std * 100 + 0.3, ret * 100, label, fontsize=11, verticalalignment='center')

        ax.text(portfolio_performance(0.18)[1]*100, 8.0, ' D', verticalalignment='center')
        ax.plot([0, 25], [8, 8], 'k--', alpha=0.7); ax.plot([0, 25], [13, 13], 'k--', alpha=0.7)
        ax.set_title('The Investment Decision', fontsize=14)
        ax.set_xlabel('Standard Deviation (%)', fontsize=12); ax.set_ylabel('Expected Return (%)', fontsize=12)
        ax.set_xlim(0, 25); ax.set_ylim(5, 14)
        ax.spines['top'].set_visible(False); ax.spines['right'].set_visible(False)
        ax.tick_params(axis='both', which='major', labelsize=11)
        plt.show()

# --- 4. Interactive Widgets Setup ---
style = {'description_width': 'initial'}
equity_weight_slider = widgets.FloatSlider(value=18.0, min=0, max=100, step=1, description='Weight in Stocks (%):', style=style, continuous_update=True)
rf_rate_slider = widgets.FloatSlider(value=5.0, min=2, max=8, step=0.25, description='Risk-Free Rate (%):', style=style, continuous_update=True)
output_info = widgets.Output(layout=widgets.Layout(overflow='auto'))
output_plot = widgets.Output()
io_link = widgets.interactive_output(generate_output, {'weight_in_equity_pct': equity_weight_slider, 'rf_rate_pct': rf_rate_slider})

# --- 5. Side-by-side Layout ---
header = widgets.HTML("<h3>Finding the Optimal Risky Portfolio</h3>")
controls = widgets.VBox([equity_weight_slider, rf_rate_slider], layout=widgets.Layout(gap='10px'))
left_col = widgets.VBox([header, controls, output_info], layout=widgets.Layout(width='45%', padding='0 12px 0 0'))
right_col = widgets.VBox([output_plot], layout=widgets.Layout(width='55%'))
ui = widgets.HBox([left_col, right_col], layout=widgets.Layout(align_items='flex-start'))

display(ui)
generate_output(equity_weight_slider.value, rf_rate_slider.value)

- **Finding the Optimal Complete Portfolio**  
$$U = E(r) - \frac{1}{2} A \sigma^2$$

---

# Lecture 6 Recap
---


## 1. The Capital Asset Pricing Model (CAPM)

### 1.1. Assumptions
- Investors are mean–variance optimizers with the same horizon, and they can borrow/lend at a risk-free rate.  
- Markets are efficient (market prices react rapidly to new information) and frictionless (no taxes/transaction costs/buying and selling restrictions).  
- All assets are tradable and divisible.  
- Homogeneous expectations: same beliefs about expected returns, variances, and covariances.

### 1.2. Main CAPM Equation

For any security \(i\):

$$
E(R_i) = r_f + \beta_i \left[ E(R_M) - r_f \right]
$$

where beta is

$$
\beta_i = \frac{\text{Cov}(R_i, R_M)}{\text{Var}(R_M)}
= \frac{\sigma_i}{\sigma_M}\rho_{i,M}
$$

- **Only systematic risk (beta)** is priced.  
- Firm-specific risk can be diversified away and **does not affect expected return**.



<center>
<img src="../BFIN_Tutorial 3/Picture4.1.png" width="700"/>

---


## 2. Security Market Line (SML) vs Capital Market Line (CML)

### 2.1. Security Market Line (SML)

Plots **expected return vs. beta**:

$$
E(R_i) = r_f + \beta_i \left[ E(R_M) - r_f \right]
$$

- Securities **above SML**: positive alpha (undervalued).  
- Securities **below SML**: negative alpha (overvalued).

### 2.2. SML vs. CML

| Line | X-axis | Y-axis | Who lies on it? | Use     |
|------|--------|--------|-----------------|--------------|
| **CML** | Total risk $ \sigma\ $ | Expected return | Only risk-free asset and market portfolio | Evaluating portfolios |
| **SML** | Systematic risk $ \beta $ | Expected return | All securities and portfolios | Evaluating individual securities |




<img src="../BFIN_Tutorial 3/Picture5.png" width="550"/>
<img src="../BFIN_Tutorial 3/Picture6.png" width="420"/>
</center>


---


## 3. Cost of Capital & Equity Risk Premium

**Cost of equity** for firm \(i\) under CAPM:

$$
k_e = r_f + \beta_i \big(R_m) - r_f \big)
$$

Long-run US historical averages (approx.):

| Metric                               | Avg. annual return (1928–2024) |
|--------------------------------------|--------------------------------:|
| S&P 500 total return                 | 11.79% |
| 3-month T-bill                       | 3.36%  |
| **Equity risk premium vs T-bill**    | **8.43%** |
| 10-year Treasury bond                | 4.79%  |
| **Equity risk premium vs 10-yr bond**| **7.00%** |

These historical premiums give a **benchmark** for the market risk premium.

---

## 4. Beta Estimation in Practice

Estimate beta using historical regressions of excess returns:

$$
(R_i - r_f) = \alpha_i + \beta_i (R_M - r_f) + e_i
$$

**Standard error of beta:**

$$
\text{S.E.}(\hat{\beta}_i) =
\frac{\sqrt{1 - R^2}}{\sqrt{T}}
\cdot
\frac{\hat{\sigma}_i}{\hat{\sigma}_M}
$$

To improve precision:

- Use a **larger sample** (\(T\)).  
- Use **portfolios** instead of individual stocks.  
- Use an appropriate **return frequency** (e.g., monthly).

---
---




# Questions

### Question 1 – Two-Asset Portfolio with Risk-Free Asset

You are given two risky assets, **P** and **Q**, and a risk-free asset:

| Asset     | Expected Return | Std. Dev. |
|-----------|-----------------|-----------|
| P         | 11%             | 18%       |
| Q         | 16%             | 28%       |
| Risk-Free | 3%              | 0%        |

The correlation between P and Q is **0.25**.

1. Find the **weights, expected return, and standard deviation** of the **Optimal Risky Portfolio (tangency portfolio)** formed by P and Q.  
2. An investor has **risk-aversion** $A = 4$ and can invest in the tangency portfolio and the risk-free asset. What is the **optimal allocation** \( y^\* \) to the risky portfolio?  
3. What are the **expected return** and **standard deviation** of this investor’s **complete portfolio**?

---

### Question 2 – $R^2$ and the Single-Index Model (Conceptual)

A security analyst runs a single-index regression of Stock **D**’s excess returns on the market’s excess returns and obtains an $R^2$ of **0.70**.

1. What does this $R^2$ tell you about the **variability of Stock D’s returns**?  
2. Does a high $R^2$ necessarily imply that Stock D will offer a **higher expected return** under the CAPM? Briefly explain.

---

### Question 3 – Single-Index Model Calculations

You are analyzing Stock **R** with the single-index model. From a regression of R’s **excess returns** on the market’s excess returns, you obtain:

- Beta $beta_R = 1.3$
- Alpha $alpha_R = 1.5\%$ per month  
- $R^2 = 0.36$
- Standard deviation of the market’s excess returns: $\sigma_M = 5\%$ per month  

1. What is the **standard deviation of Stock R’s excess returns**?  
2. What is the **covariance** between Stock R and the market?  
3. What is the **correlation coefficient** between Stock R and the market?

---

### Question 4 – CAPM vs Total Risk (Conceptual)

The **CAPM** states that the expected return of a security depends only on its **systematic risk** (beta), not on its total standard deviation. Yet, when constructing an optimal risky portfolio with **Markowitz portfolio theory**, the **full covariance matrix of returns** is required.

Explain how these two ideas are **reconciled**.

---

### Question 5 – Two Stocks with Single-Index Regressions

An analyst estimates the following single-index regressions of excess returns on the market’s excess returns:

| Stock | Alpha \( \alpha \) | Beta \( \beta \) | Residual Std. Dev. \( \sigma_\varepsilon \) |
|-------|--------------------|------------------|----------------------------------------------|
| A     | 1.2%               | 1.1              | 7%                                           |
| B     | –0.4%              | 0.9              | 5%                                           |

The expected excess return on the market is \( E(R_M) = 8\% \) and the standard deviation of the market’s excess returns is \( \sigma_M = 18\% \).

1. What are the **expected excess returns** for Stocks A and B?  
2. What is the **covariance** between the returns of Stocks A and B?  
3. Consider an **equally weighted** portfolio of A and B. What are this portfolio’s:
   - alpha,  
   - beta,  
   - residual standard deviation, and  
   - total standard deviation?

---

### Question 6 – Active Portfolio and Kappa Corp

You are evaluating **Kappa Corp** for potential mispricing using the CAPM. You have:

- Kappa Corp’s beta: $ \beta_K $ = 1.4
- Kappa’s residual standard deviation: $ \sigma_{\varepsilon,K} = 12\% $
- Your forecast for Kappa’s expected return: $ E(R_K) = 18\% $
- Your forecast for the market return: $$ E(R_M) = 11\% $
- Market standard deviation: $ \sigma_M = 19\% $
- Risk-free rate: $ r_f = 3\% $

You plan to form an **optimal active portfolio** by combining Kappa Corp with the passive market index.

1. According to the CAPM, what is the **required rate of return** for Kappa Corp?  
2. What is Kappa’s **alpha** based on your forecasts?  
3. Compute the **optimal weight assigned to Kappa**, $ w_A^* $, in the risky active portfolio of Kappa and the market, using  
   $$
      w_A^* = \frac{\alpha / \sigma_{\varepsilon,K}^2}{\left(E(R_M) - r_f\right)/\sigma_M^2}
   $$  
4. What is the **beta of the new optimal risky portfolio** formed by Kappa and the market?  
5. An analyst claims that because the new portfolio’s beta is greater than 1, its **Sharpe ratio must be lower than the market’s**. Is this correct? Compute the **Sharpe ratio** of the new portfolio to justify your answer.

---

### Question 7 - Minimum Variance Frontier
- Why does allowing short shift the MV Frontier to the left?
